In [ ]:
library("ArchR")
library("GenomicRanges")
library('BSgenome')
library('TxDb.Rnorvegicus.UCSC.rn6.refGene')
library('BSgenome.Rnorvegicus.UCSC.rn6')
library("org.Rn.eg.db")
library("dplyr")                                    # Load dplyr package
library("plyr")                                     # Load plyr package
library("readr")  
library(qdap)
library('Seurat')

In [ ]:
proj3 <- loadArchRProject(path = ".", force = FALSE, showLogo = TRUE)
proj3

In [ ]:
getCellColData(proj3)$Condition

# Identifying Marker Genes

In [ ]:
############------------------------Identifying Marker Genes--------------------------###################

markersGS <- getMarkerFeatures(
  ArchRProj = proj3, 
  useMatrix = "GeneScoreMatrix",
  groupBy = "Clusters",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "ttest",#"wilcoxon"

)

In [ ]:
# save for shiny app

saveRDS(markersGS,"markersGS_clusters.rds")

In [ ]:
# save for shiny

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 0.20", plotLog2FC = TRUE,
#   labelMarkers = seleceted_markers,
  transpose = F,  returnMatrix = TRUE
)

write.csv(heatmapGS,"genes_per_cluster_hm.csv")

In [ ]:
heatmapGS

In [ ]:
# per sample

######------------------------------Identifying Marker Genes--------------------------------#######

markersGS <- getMarkerFeatures(
  ArchRProj = proj3, 
  useMatrix = "GeneScoreMatrix", 
  groupBy = "Sample",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "ttest",#"wilcoxon"
)

# save for shiny app
saveRDS(markersGS,"markersGS_sample.rds")


proj3 <- addImputeWeights(proj3,reducedDims = "Harmony")                     
                     
options(repr.plot.width=7, repr.plot.height=7)

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 0.20", plotLog2FC = TRUE,
#   labelMarkers = seleceted_markers,
  transpose = F,  returnMatrix = FALSE
)
    
heatmapGS
                     
                     
                     
# save for shiny

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 0.20", plotLog2FC = TRUE,
#   labelMarkers = seleceted_markers,
  transpose = F,  returnMatrix = TRUE
)
write.csv(heatmapGS,"genes_per_sample_hm.csv")                     

In [ ]:
# per treatment

######------------------------------Identifying Marker Genes--------------------------------#######

markersGS <- getMarkerFeatures(
  ArchRProj = proj3, 
  useMatrix = "GeneScoreMatrix", 
  groupBy = "Condition",
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "ttest",#"wilcoxon"
)
# save for shiny app
saveRDS(markersGS,"markersGS_treatment.rds")


proj3 <- addImputeWeights(proj3,reducedDims = "Harmony")                     
                     
options(repr.plot.width=7, repr.plot.height=7)

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 0.20", plotLog2FC = TRUE,
#   labelMarkers = seleceted_markers,
  transpose = F,  returnMatrix = FALSE
)
    
heatmapGS
                     
                     
                     
# save for shiny

heatmapGS <- plotMarkerHeatmap(
  seMarker = markersGS, 
  cutOff = "FDR <= 0.05 & Log2FC >= 0.20", plotLog2FC = TRUE,
#   labelMarkers = seleceted_markers,
  transpose = F,  returnMatrix = TRUE
)
write.csv(heatmapGS,"genes_per_treatment_hm.csv")                     

# Volcano plots for genes

In [ ]:
# for all clusters together

In [ ]:
  ncells <- length(proj3$cellNames)

  markerList <- getMarkerFeatures(
    ArchRProj = proj3,
    useMatrix = "GeneScoreMatrix", 
    groupBy = "Condition",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    maxCells = ncells,
    normBy = "none",
    testMethod = "ttest"
)

In [ ]:


p4<-plotEmbedding(ArchRProj = proj3, colorBy = "cellColData", name = "Clusters", embedding = "UMAP"
                  ,size=1.0, baseSize = 10)+
            theme(legend.position = "top",legend.direction= "horizontal"
                , legend.text=element_text(size=20), legend.title=element_text(size=0))+
   theme(
  plot.title = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank())
# p4
p5<-plotEmbedding(ArchRProj = proj3, colorBy = "cellColData", name = "Sample", embedding = "HA"
                  ,size=1.0, baseSize = 10)+
            theme(legend.position = "top",legend.direction= "vertical"
                , legend.text=element_text(size=20), legend.title=element_text(size=0))+
   theme(
  plot.title = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank())
# p5

p6<-plotEmbedding(ArchRProj = proj3, colorBy = "cellColData", name = "Condition", embedding = "UMAP"
                  ,size=1.0, baseSize = 10)+
            theme(legend.position = "top",legend.direction= "vertical"
                , legend.text=element_text(size=20), legend.title=element_text(size=0))+
   theme(
  plot.title = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank())
# p6


options(repr.plot.width=15, repr.plot.height=9)

p4 + p5 + p6 

In [ ]:
pal <- paletteDiscrete(values = getCellColData(proj3)$Condition)


options(repr.plot.width=12, repr.plot.height=7)

req_DF <- as.data.frame(getCellColData(proj3))

req_table <- melt(table(req_DF$Clusters,req_DF$Condition))
colnames(req_table) <- c("Cluster","Treatment","%cells in knn clusters")
req_table$Cluster <- factor(req_table$Cluster
                        , levels = (unique(req_table[order(as.numeric(gsub("C","",req_table$Cluster))),]$Cluster)))

ggplot(req_table, aes(fill=Treatment, y=`%cells in knn clusters`, x= Cluster))+ geom_bar(stat="identity"
                                                                                      , position = "fill")+
theme_classic() + theme(text = element_text(size = 25)) +theme(axis.title.x=element_blank(),)+
  scale_fill_manual(values= (pal))

# Running Harmony Again!

In [ ]:
proj4 <- proj3

In [ ]:
########-----------------------------Batch Effect Correction wtih Harmony----------------########
proj4 <- addHarmony(
    ArchRProj = proj4,
    reducedDims = "IterativeLSI",
    name = "Harmony",
    groupBy = "Sample",
    force = TRUE
                      )

In [ ]:
proj4 <- addClusters(
    input = proj4,
    reducedDims = "Harmony",
    method = "Seurat",
    name = "Clusters2",
#     resolution = 0.5,
    resolution = 1.5,
    force = TRUE
                    )

In [ ]:
proj4 <- addUMAP(
    ArchRProj = proj4, 
    reducedDims = "Harmony", 
    name = "UMAPHarmony", 
    nNeighbors = 20, 
    minDist = 0.5, 
    metric = "cosine",verbose = FALSE,
    ,force = TRUE
                 )

In [ ]:
pal <- paletteDiscrete(values = getCellColData(proj4)$Condition)


options(repr.plot.width=12, repr.plot.height=7)

req_DF <- as.data.frame(getCellColData(proj4))

req_table <- melt(table(req_DF$Clusters2,req_DF$Condition))
colnames(req_table) <- c("Cluster","Treatment","%cells in knn clusters")
req_table$Cluster <- factor(req_table$Cluster
                        , levels = (unique(req_table[order(as.numeric(gsub("C","",req_table$Cluster))),]$Cluster)))

ggplot(req_table, aes(fill=Treatment, y=`%cells in knn clusters`, x= Cluster))+ geom_bar(stat="identity"
                                                                                      , position = "fill")+
theme_classic() + theme(text = element_text(size = 25)) +theme(axis.title.x=element_blank(),)+
  scale_fill_manual(values= (pal))

### A few scripts for finding automatically dominated clusters by one of the conditions

In [ ]:
df1 <- table(req_DF$Clusters2,req_DF$Condition)

distr <- as.data.frame.matrix(round(prop.table(as.matrix(df1),1),2))

distr

In [ ]:
lst <- list()

for(i in 1:nrow(distr)) {
    row <- distr[i,]
    if (
    sum(unname(unlist(row))>= 0.85) == 1) {
        rownames(row) -> lst[[i]]
    }
}
not_req_list <- unlist(lst)
not_req_list

In [ ]:
# the above bar plot shows that all of the clusters (EXCEPT 9) have a minimum of each treatment so we don't get 
# error "2 function calls resulted in an error" and we run volcanon for all of the clusters

req_clusters <- unique(proj4$Clusters2)
req_clusters <- req_clusters[order(as.numeric(gsub("C","",req_clusters)))]


# remove cluster 9
req_clusters <- req_clusters[which(!req_clusters%in%not_req_list)]

req_clusters

In [ ]:
req_clusters[1]

In [ ]:

######------------------Identifying Marker Genes grouped by treatment per each cluster -----------------------#######

markerList_C <- list()
proj4_C <- list()

for (i in seq_along(req_clusters)) {
    
   idxSample <- BiocGenerics::which(proj4$Clusters2 == req_clusters[i])
   
   cellsSample <- proj4$cellNames[idxSample]
   proj4_C[i] <- proj4[cellsSample,]
    
    ncells[i] <- length(proj4_C[[i]]$cellNames)

# per each cluster separately
  markerList_C[[i]] <- getMarkerFeatures(
  ArchRProj = proj4_C[[i]],
  useMatrix = "GeneScoreMatrix", 
  groupBy = "Condition",
  bias = c("TSSEnrichment", "log10(nFrags)"),maxCells = ncells[[i]] ,normBy = "none",
  testMethod = "ttest")
    

    }

names(markerList_C) <- req_clusters

In [ ]:
# lets find empty genes

gsm <- getMatrixFromProject(proj4)
gsm_mat <- assay(getMatrixFromProject(proj4),"GeneScoreMatrix")

which(rowSums(is.na(gsm_mat))>0)
any(rowSums((gsm_mat))==0)

empty_gene_idx <- which(rowSums((gsm_mat))==0)
empty_gene <- rowData(gsm)$name[empty_gene_idx]
empty_gene

In [ ]:
colnames(markerList)

In [ ]:
# volcano data for All clusters together

In [ ]:
markerList
assays(markerList)

In [ ]:
grep('Ptprg',rowData(markerList)$name)

In [ ]:
assay(markerList, "Log2FC")[6155,]

In [ ]:
markerList_df1 <- assay(markerList, "Log2FC")
markerList_df2 <- assay(markerList, "Pval")
markerList_df3 <- assay(markerList, "FDR")
markerList_df <- cbind(markerList_df1,markerList_df2,markerList_df3)
markerList_df$genes<- rowData(markerList)$name
markerList_df$cluster <- rep("All",length(rownames(markerList_df)))

# # we only want to see results of one set , say just sham
markerList_df <- markerList_df[,c(1,3,5,7,8)]
colnames(markerList_df) <- c("avg_log2FC","p_val","p_val_adj","gene","cluster")

# # it happens alot that fdr values are similar. for those cases we only keep fdrs with highest logfc
# markerList_df$logfdr <- -log10(markerList_df$p_val_adj)
# markerList_df <- setDT(markerList_df)[order(-abs(avg_log2FC)), .SD[1L] ,.(logfdr)]


markerList_df

In [ ]:
table(markerList_df$cluster)

In [ ]:
# volcano data per cluster separately

In [ ]:
req_clusters

### Here I changed naming of the clusters

In [ ]:

markerList_df1_C <- list()
markerList_df2_C <- list()
markerList_df3_C <- list()
markerList_df_C <- list()

# for (i in (1:nClust)){
for (i in seq_along(req_clusters)){
    
cluster <- req_clusters[i]
    
markerList_df1_C[[i]] <- assay(markerList_C[[i]], "Log2FC")
markerList_df2_C[[i]] <- assay(markerList_C[[i]], "Pval")
markerList_df3_C[[i]] <- assay(markerList_C[[i]], "FDR")
    
markerList_df_C[[i]] <- cbind(markerList_df1_C[[i]]
                              ,markerList_df2_C[[i]]
                              ,markerList_df3_C[[i]])
    
markerList_df_C[[i]]$genes<- rowData(markerList_C[[i]])$name
markerList_df_C[[i]]$cluster <- rep(cluster,length(rownames(markerList_df_C[[i]])))

# # we only want to see results of one set , say just sham
markerList_df_C[[i]] <- markerList_df_C[[i]][,c(1,3,5,7,8)]
colnames(markerList_df_C[[i]]) <- c("avg_log2FC","p_val","p_val_adj","gene","cluster")

    
# # it happens alot that fdr values are similar
# # we only keep fdrs with highest logfc
# markerList_df_C[[i]]$logfdr <- -log10(markerList_df_C[[i]]$p_val_adj)
# markerList_df_C[[i]] <- setDT(markerList_df_C[[i]])[order(-abs(avg_log2FC)), .SD[1L] ,.(logfdr)]

    }


names(markerList_df_C) <- req_clusters

In [ ]:
for (i in seq_along(req_clusters)){
    
print(req_clusters[i])
    }

In [ ]:
#### merge all data frames

markersGS_merged_df <- do.call("rbind", markerList_df_C)

# also data frame for all clusters together needs to be added

markersGS_merged_df <- rbind(markerList_df,markersGS_merged_df)

In [ ]:
# remove empty genes
markersGS_merged_df <- markersGS_merged_df[which(!markersGS_merged_df$gene%in%empty_gene),]

# remove na values
markersGS_merged_df <- na.omit(markersGS_merged_df)

# remove FDR equal to 0
markersGS_merged_df <- markersGS_merged_df[which(!markersGS_merged_df$p_val_adj== 0),]



# make logfc limiation between 1 and -1

markersGS_merged_df <- markersGS_merged_df[which(abs(markersGS_merged_df$avg_log2FC)< 1.2),]

markersGS_merged_df$Significance = ifelse(markersGS_merged_df$p_val_adj < 10^-1 , 
#                                           abs(markersGS_merged_df$avg_log2FC) >= 0.58, 
                     ifelse(markersGS_merged_df$avg_log2FC> 0.0 
                            ,colnames(markerList)[1],colnames(markerList)[2]),
                     'Not siginficant')
markersGS_merged_df 

In [ ]:
table(markersGS_merged_df$cluster)

In [ ]:
de <- markersGS_merged_df
table(de$Significance)

In [ ]:
write.table(de,"inpMarkers.txt", sep = '\t', quote = F, row.names = F)

In [ ]:
inpMarkers = fread("./inpMarkers.txt") 
# inpMarkers

In [ ]:
# let's test
library(ggplot2)
library(grid)
library(ggrepel)
library(ggpubr)

In [ ]:
targeted <- 'All'

#   minfdr = 10^-(1/3 *(-log10(min(inpMarkers[cluster == targeted]$p_val_adj))))
    minfdr = 0.05
  minfdr2 = 10^-(2/3 *(-log10(min(inpMarkers[cluster == targeted]$p_val_adj))))

ggData = inpMarkers[cluster == targeted]
ggData$Significance = ifelse(ggData$p_val_adj < minfdr ,
#                              & abs(ggData$avg_log2FC) >= 0.58,
                                 ifelse(ggData$avg_log2FC > 0.0
                                        ,"Lupus","WT")
                                 ,'Not siginficant'
    )


ggData$Significance <- factor(
                               ggData$Significance,
                               levels = c('Lupus','WT','Not siginficant')
                             )


    ggData[p_val_adj < 1e-300]$p_val_adj = 1e-300
    ggData$log10fdr = -log10(ggData$p_val_adj)
    ggplot(ggData, aes(avg_log2FC, log10fdr)) + 
    geom_point() + 
# sctheme() + 
    ylab("-log10(FDR)") +

    geom_point(aes(color = Significance)) +

    scale_color_manual(values = c("red","blue","grey")) +

    geom_text_repel(
    data = subset(ggData, p_val_adj < 10^-5 ),
    aes(label = gene))

# for Shiny app

In [ ]:
# Extract umap from project

In [ ]:
names(proj3@reducedDims$Harmony@listData)

In [ ]:
plotEmbedding(ArchRProj = proj3, colorBy = "cellColData", name = "Clusters_edited", embedding = "UMAPHarmony"
                  ,size=1.0, baseSize = 10)+
            theme(legend.position = "right",legend.direction= "vertical"
                , legend.text=element_text(size=20), legend.title=element_text(size=0))+
   theme(
  plot.title = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank())
# p4

In [ ]:
UMAPHarmony <-getEmbedding(ArchRProj = proj3, embedding = "UMAPHarmony", returnDF = TRUE)
UMAPHarmony

In [ ]:
write.csv(UMAPHarmony,"UMAPHarmony.csv")

In [ ]:
# Motif Logo

library("seqLogo")
require(ggseqlogo)
library(ArchR)
library(chromVARmotifs)

# data("human_pwms_v1")

PWMs <- getPeakAnnotation(proj3, "Motif")$motifs

PWMatrixToProbMatrix <- function(x){
	if (class(x) != "PWMatrix") stop("x must be a TFBSTools::PWMatrix object")
	(exp(as(x, "matrix"))) * TFBSTools::bg(x)/sum(TFBSTools::bg(x))
}
 
ProbMatrices <- lapply(PWMs, PWMatrixToProbMatrix)
lapply(ProbMatrices, colSums) %>% range
#[1] 0.9999996 1.0000004

#Maybe we can just tidy this up a tiny bit

PWMatrixToProbMatrix <- function(x){
	if (class(x) != "PWMatrix") stop("x must be a TFBSTools::PWMatrix object")
	m <- (exp(as(x, "matrix"))) * TFBSTools::bg(x)/sum(TFBSTools::bg(x))
	m <- t(t(m)/colSums(m))
	m
}

ProbMatrices <- lapply(PWMs, PWMatrixToProbMatrix)
lapply(ProbMatrices, colSums) %>% range
#[1] 1 1

In [ ]:
grep("AP1_714", names(ProbMatrices))

In [ ]:

ggseqlogo(ProbMatrices[c(grep(paste0("^","AP1_714","$"), names(ProbMatrices)))])


In [ ]:
saveRDS(ProbMatrices,"seqlogo.rds")

# Default genes for heatmaps

In [ ]:
suppressPackageStartupMessages(library("ComplexHeatmap"))
suppressPackageStartupMessages(library("circlize"))
suppressPackageStartupMessages(library(data.table))

In [ ]:
hm_per_clust <- read.csv("genes_per_cluster_hm.csv")
# hm_per_clust
hm_per_sample <- read.csv("genes_per_sample_hm.csv")
# hm_per_sample
hm_per_cond <- read.csv("genes_per_treatment_hm.csv")

In [ ]:
nClust = 9


df = list()


for (i in seq_along(1:nClust)){
df[[i]] <- hm_per_clust[,c(1,i+1)]

#select top 5 values by group

df[[i]] <- df[[i]][order(df[[i]][,2], decreasing = T),][1:5,1]

}
final <- do.call(rbind, df)
final

In [ ]:
req_genes1 <- unlist(df)

req_genes1<- req_genes1[!duplicated(req_genes1)]

# save the genes for default values in shiny app

write.csv(req_genes1,"req_genes1.csv")

In [ ]:
# per treatment

nConds = 2


df = list()

for (i in seq_along(1:nConds)){
df[[i]] <- hm_per_cond[,c(1,i+1)]

#select top 20 values by group

df[[i]] <- df[[i]][order(df[[i]][,2], decreasing = T),][1:20,1]

}
final <- do.call(rbind, df)
# final
req_genes2 <- unlist(df)

req_genes2<- req_genes2[!duplicated(req_genes2)]

write.csv(req_genes2,"req_genes2.csv")

In [ ]:
# per sample

nSamples = 4


df = list()

for (i in seq_along(1:nSamples)){
df[[i]] <- hm_per_sample[,c(1,i+1)]

#select top 10 values by group

df[[i]] <- df[[i]][order(df[[i]][,2], decreasing = T),][1:10,1]

}
final <- do.call(rbind, df)
# final
req_genes3 <- unlist(df)

req_genes3<- req_genes3[!duplicated(req_genes3)]

write.csv(req_genes3,"req_genes3.csv")